### 0. 連線到我的 Google Drive 獲取資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/nlp-getting-started

/content/drive/MyDrive/nlp-getting-started


In [ ]:
import pandas as pd 
df = pd.read_csv('./train.csv')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. 使用 Keras 來建立一個 Trasformer Input Block
https://github.com/keras-team/keras-io/blob/master/examples/nlp/text_classification_with_transformer.py

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [ ]:
token_set = set()
for text in df['text'].values:
  for token in text.split():
    token_set.add(token)

In [ ]:
vocab_size = len(token_set)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
num_words = 20000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [ ]:
# Tokenize our training data
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(df['text'].values)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(df['text'].values)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
print("Word index:\n", word_index)
print("\nTraining sequences:\n", train_sequences)
print('\nMax Length: \n', maxlen)
print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

Word index:
 {'<UNK>': 1, 't': 2, 'co': 3, 'http': 4, 'the': 5, 'a': 6, 'in': 7, 'to': 8, 'of': 9, 'and': 10, 'i': 11, 'is': 12, 'for': 13, 'on': 14, 'you': 15, 'my': 16, 'with': 17, 'that': 18, 'it': 19, 'at': 20, 'by': 21, 'this': 22, 'from': 23, 'https': 24, 'are': 25, 'be': 26, 'was': 27, 'have': 28, 'like': 29, 'amp': 30, 'as': 31, 'up': 32, 'me': 33, 'but': 34, 'just': 35, 'so': 36, 'not': 37, 'your': 38, 'out': 39, 'no': 40, 'all': 41, 'after': 42, 'will': 43, 'an': 44, 'has': 45, 'fire': 46, "i'm": 47, 'when': 48, 'if': 49, 'we': 50, 'get': 51, 'now': 52, 'new': 53, 'via': 54, 'more': 55, '2': 56, 'about': 57, 'or': 58, 'news': 59, 'people': 60, 'what': 61, 'they': 62, 'one': 63, 'he': 64, 'how': 65, 'been': 66, 'over': 67, 'who': 68, "it's": 69, 'into': 70, "don't": 71, 'do': 72, 'video': 73, "'": 74, 'can': 75, 'emergency': 76, 'disaster': 77, 'there': 78, 'police': 79, 'than': 80, '3': 81, 'her': 82, 'u': 83, 'would': 84, 'still': 85, 'his': 86, 'some': 87, 'were': 88, 'body

In [ ]:
embed_dim = 8  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 4  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(8, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
import numpy as np
def shuffled_data(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
split_ratio = 0.8
shuffle_x, shuffle_y = shuffled_data(train_padded, df['target'].values)
split_idx = int(len(shuffle_x) * split_ratio)
train_x, train_y, val_x, val_y = shuffle_x[:split_idx], shuffle_y[:split_idx], shuffle_x[split_idx:], shuffle_y[split_idx:]

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    train_x, train_y, batch_size=256, epochs=10, validation_data=(val_x, val_y)
)

Epoch 1/10
24/24 [==============================] - 4s 30ms/step - loss: 0.6891 - accuracy: 0.5644 - val_loss: 0.6864 - val_accuracy: 0.6001
Epoch 2/10
24/24 [==============================] - 0s 10ms/step - loss: 0.6750 - accuracy: 0.5856 - val_loss: 0.6690 - val_accuracy: 0.6146
Epoch 3/10
24/24 [==============================] - 0s 10ms/step - loss: 0.6449 - accuracy: 0.6314 - val_loss: 0.6168 - val_accuracy: 0.7091
Epoch 4/10
24/24 [==============================] - 0s 9ms/step - loss: 0.5938 - accuracy: 0.6737 - val_loss: 0.5602 - val_accuracy: 0.7584
Epoch 5/10
24/24 [==============================] - 0s 10ms/step - loss: 0.5220 - accuracy: 0.7279 - val_loss: 0.4992 - val_accuracy: 0.7800
Epoch 6/10
24/24 [==============================] - 0s 9ms/step - loss: 0.4581 - accuracy: 0.7583 - val_loss: 0.4755 - val_accuracy: 0.7919
Epoch 7/10
24/24 [==============================] - 0s 9ms/step - loss: 0.3867 - accuracy: 0.7997 - val_loss: 0.4883 - val_accuracy: 0.7951
Epoch 8/10
24/24